In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('open_engageny_decoded.csv', encoding='latin1')
len(df)

46373

In [3]:
df_teach_to_assignment = pd.read_csv('teacher_to_assignments.csv', 
                                     encoding='latin1', sep=';')
removelist = []
removeteachers = []
df_teach_to_assignment.head()
for teacherid in set(df_teach_to_assignment.teacher_id):
    target_as_ids = df_teach_to_assignment[
        df_teach_to_assignment['teacher_id'].isin([teacherid])]
    target_problems = df[df['assignment_id'].isin(target_as_ids.id)]
    # get score distributions
    scores = target_problems.correct.values.tolist()
    score_dist = [scores.count(0), scores.count(0.25), 
                  scores.count(0.5), scores.count(0.75), scores.count(1)]
    if sum(score_dist) > 0:
        score_dist = np.array(score_dist) / sum(score_dist)
        var = np.var(score_dist)

        if var > 0.1:
            removelist.extend(list(set(target_as_ids.id)))

df = df[~df['assignment_id'].isin(removelist)]
len(df)

38773

In [4]:
import re

removelist = []
for pid in set(df.problem_id):
    target_problems = df[df['problem_id'].isin([pid])]
    problem_count = len(target_problems)
    ans_texts = target_problems.answer_text.values.tolist()
    # clean up string so that we can do proper comparisons
    ans_texts = [str(ans).lower() for ans in ans_texts]
    ans_texts = [re.sub(r'[â(),.!?\-\+\=\/\*\'\`]', '', ans) for ans in ans_texts]
    ans_texts = [re.sub('\n', '', ans) for ans in ans_texts]
    ans_texts = [''.join(ans.split()) for ans in ans_texts]
    ans_counts = []
    # count number of identical answers
    for ans in set(ans_texts):
        ans_counts.append(ans_texts.count(ans))
    # if identical answer count for any particular answer > 10 or 25% of the responses, get rid of it.
    # the exact cutoffs are slightly arbitrary
    if max(ans_counts) >= int(problem_count/4):
        removelist.append(pid)
        
df = df[~df['problem_id'].isin(removelist)]
len(df)

6996

In [5]:
removelist = []
for pid in set(df.problem_id):
    target_problems = df[df['problem_id'].isin([pid])]
    problem_count = len(target_problems)
    ans_texts = target_problems.answer_text.values.tolist()
    # responses that contain little to no alphabet characters are probably problems that require a
    # single number / equation for the answer. count unique characters to detect number of responses that
    # match this situation.
    low_alpha_count = 0
    for ans in ans_texts:
        if isinstance(ans, float):
            low_alpha_count += 1
        else:
            ans_chars = []
            for c in ans.lower():
                if c.isalpha():
                    ans_chars.append(c)
            ans_chars = list(set(ans_chars))
            if len(ans_chars) < 4:
                low_alpha_count += 1
    # if >25% of responses contain little to no alphabet characters, remove.
    if low_alpha_count > problem_count / 4:
        removelist.append(pid)
df = df[~df['problem_id'].isin(removelist)]
len(df)

4877

In [6]:
import data_utils
df.reset_index(drop=True)
df_original = df
# remove answers with scores that aren't regular
df = df[(df['correct'] % 0.25 == 0)]
# remove answers that are effectively blank (after applying some cleaning, so that answers like "\n" can properly get filtered out
df['answer_text'] = df['answer_text'].apply(lambda x: data_utils.clean_str(x))
df = df[(df['answer_text'] != '')]
df = df[(df['answer_text'] != 'nan')]
# just doing this so that the string cleaning isn't saved for the final dataset
df = df_original[df_original.index.isin(df.index)]
len(df)

E:\Python\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4701

In [ ]:
removelist = []
for pid in set(df.problem_id):
    target_problems = df[df['problem_id'].isin([pid])]
    grades = set(target_problems.correct)
    if len(grades) == 1:
        removelist.append(pid)
        print(pid)
df = df[~df['problem_id'].isin(removelist)]
len(df)

In [ ]:
min_count = 10

# filter by response count, >10
df_counts_all = df['problem_id'].value_counts()
df_counts = df[df['problem_id'].isin(df_counts_all.index[df_counts_all > min_count]).values].reset_index()

df = df_counts
len(df)

4649

In [8]:
df.to_csv('filter_engageny_openresponse.csv')
len(set(df.problem_id))

173